# plot imgs or feature maps

In [ ]:
import matplotlib.pyplot as plt
import cv2
idx=0
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 5))  # Adjust figsize as needed

# Display the first image
ax1.imshow(cv2.resize(imgs[idx].permute(1,2,0).detach().cpu().numpy(), (128, 128)))
ax1.set_title('Image 1')
ax1.axis('off')  # Turn off axes for cleaner presentation (optional)

# Display the second image
ax2.imshow(cv2.resize(positive_imgs[idx].permute(1,2,0).detach().cpu().numpy(), (128, 128)))
ax2.set_title('Image 2')
ax2.axis('off')  # Turn off axes (optional)

ax3.imshow(cv2.resize(negative_imgs[idx].permute(1,2,0).detach().cpu().numpy(), (128, 128)))
ax3.set_title('Image 3')
ax3.axis('off')  # Turn off axes (optional)

# Adjust spacing between subplots (optional)
plt.subplots_adjust(left=0.05, right=0.95, wspace=0.05)

In [ ]:
import pandas as pd
import os

# Define a list to store the dataframes
all_dfs = []

# Loop through the CSV files and read them into dataframes
for filename in os.listdir('./csv_results/report_class_1/'):  # Replace with your filenames
  df = pd.read_csv(filename)
  all_dfs.append(df)

# Combine all dataframes vertically (stacking rows)
combined_df = pd.concat(all_dfs, ignore_index=True)  # Optional: reset index

# Save the combined dataframe to a new CSV file
combined_df.to_csv("combined.csv", index=False)  # Optional: remove index column


In [ ]:
import pandas as pd
import os
all_dfs = []
names= []
root='./csv_results/report_class_1/'
for filename in os.listdir(root):
    print(filename)
    df = pd.read_csv(root + filename)
    df.insert(loc=0, column='noise', value=filename.split('.')[0])
    all_dfs.append(df)
    names.append(filename.split('.')[0])

combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df.to_csv(root + "combined.csv", index=False)

In [ ]:
import os
import torch
import pickle
import numpy as np

dataset = 'cifar10'

softmax_sorted = {}
diff_vals = {}
for class_idx in range(10):
    diff_vals[class_idx] = {}

targets_list_loaded = np.load('/storage/users/makhavan/CSI/finals/datasets/generalization_repo_dataset/manual/CIFAR-10-Train-R-A/labels-A.npy')
for class_idx in range(10):
    indices = [k for k in range(len(targets_list_loaded)) if targets_list_loaded[k]==class_idx]
    for file_name in os.listdir('./clip_vec/tensors/'):
        if dataset in file_name and 'target' not in file_name and 'normal_data' not in file_name:
            with open('./clip_vec/tensors/' + file_name, 'rb') as file:
                diffs_loaded = pickle.load(file)
        
            noise_name = file_name.replace(f"{dataset}_diffs_", "", 1).split('.')[0]
            diff_vals[class_idx][noise_name] = torch.mean(torch.tensor([diffs_loaded[idx].astype(np.float32) for idx in indices])).float()

for i in range(10):
    softmaxes = torch.nn.functional.softmax(torch.tensor(list(diff_vals[i].values())), dim=0).numpy()
    for j, key in enumerate(diff_vals[i].keys()):
        diff_vals[i][key] = softmaxes[j]

for i in range(10):
    sorted_values = sorted(diff_vals[i].items(), key=lambda item: item[1])
    diff_vals[i] = dict(sorted_values)


In [ ]:
import os
import torch
import pickle
import numpy as np

dataset = 'svhn'

softmax_sorted = {}
diff_vals = {}
for class_idx in range(10):
    diff_vals[class_idx] = {}

targets_list_loaded = np.load('/storage/users/makhavan/CSI/finals/datasets/generalization_repo_dataset/manual/SVHN-Train-R-A/labels-A.npy')
for class_idx in range(10):
    indices = [k for k in range(len(targets_list_loaded)) if targets_list_loaded[k]==class_idx]
    for file_name in os.listdir('./clip_vec/tensors/'):
        if dataset in file_name and 'target' not in file_name and 'normal_data' not in file_name:
            with open('./clip_vec/tensors/' + file_name, 'rb') as file:
                diffs_loaded = pickle.load(file)
        
            noise_name = file_name.replace(f"{dataset}_diffs_", "", 1).split('.')[0]
            diff_vals[class_idx][noise_name] = torch.mean(torch.tensor([diffs_loaded[idx].astype(np.float32) for idx in indices])).float()

for i in range(10):
    softmaxes = torch.nn.functional.softmax(torch.tensor(list(diff_vals[i].values())), dim=0).numpy()
    for j, key in enumerate(diff_vals[i].keys()):
        diff_vals[i][key] = softmaxes[j]

for i in range(10):
    sorted_values = sorted(diff_vals[i].items(), key=lambda item: item[1])
    diff_vals[i] = dict(sorted_values)


In [ ]:
with open('./svhn_softmaxed.pkl', 'wb') as f:
    pickle.dump(diff_vals, f)

In [ ]:
import pandas as pd
dfs = []
for i in range(10):
    pd.DataFrame(diff_vals[i], index=[i]).T.to_csv(f'{i}.csv', index=True)